In [26]:
# imports 
import torch 
import torch.nn as nn
from torch.nn import functional as F

In [5]:
# read the input.txt
with open('input.txt', 'r') as file:
    text = file.read()

In [6]:
f'length of dataset (characters): {len(text)}'

'length of dataset (characters): 1115394'

In [7]:
# first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [8]:
# unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
f'vocabulary size: {vocab_size}'


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


'vocabulary size: 65'

In [9]:
# characters mapping to integers 
stoi = {ch: i for i, ch in enumerate(chars)}  # create a dictionary that maps characters to their integer indices
itos = {i: ch for i, ch in enumerate(chars)}  # create a dictionary that maps integer indices to their corresponding characters
def encode(s):
    return [stoi[c] for c in s]  # function to encode a string into a list of integers  
def decode(encoded_list):
    return "".join(
        itos[i] for i in encoded_list
    )  # function to decode a list of integers into a string


encode('hi there'), decode(encode('hi there'))

([46, 47, 1, 58, 46, 43, 56, 43], 'hi there')

In [10]:
# encode the entire dataset using pytorch 
data = torch.tensor(encode(text), dtype=torch.long) # encode the text into tensor of long integers

data.shape, data.dtype, data[:1000]

(torch.Size([1115394]),
 torch.int64,
 tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
         53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
          1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
         57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
          6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
         58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
          1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
         53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
         57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
          8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
          1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
         53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39

In [11]:
# train & validation split 
n = int(0.9 * len(data)) 
train_data = data[:n]
valid_data = data[n:]

In [20]:
# a sequence of text with block_size + 1 characters
block_size = 8
print(train_data[:block_size + 1])
print('\n')

x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f'when input is {context}, the target is {target}')


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])


when input is tensor([18]), the target is 47
when input is tensor([18, 47]), the target is 56
when input is tensor([18, 47, 56]), the target is 57
when input is tensor([18, 47, 56, 57]), the target is 58
when input is tensor([18, 47, 56, 57, 58]), the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]), the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is 58


In [21]:
'''
This code snippet is initializing variables and defining a function to generate batches of training data for a neural network model.

The purpose is to create batches of input data x and target output data y to train the model. It takes no explicit inputs. The key outputs are:

xb - a batch of input data
yb - the target outputs corresponding to xb
It achieves this by first setting some hyperparameters:

batch_size - the number of data sequences in each batch
block_size - the maximum length of each sequence
It then defines a get_batch() function that takes a 'train' or 'val' split. This function samples random start indices ix from the full dataset, extracts sequences of length block_size starting from those indices, and stacks them into the x and y tensors.

The key logic flow is:

Sample random indices
Slice sequences of block_size from the dataset at those indices
Stack the sequences into batches x and y
This creates batches of data for training or validation, where y are the target outputs for input sequences x. The batches contain multiple independent sequences of limited length.

Finally, the function is called to generate a sample batch and print out the shape and values, showing how the batches are constructed.

'''
torch.manual_seed(19)
batch_size = 4 # number of sequences in a batch
block_size = 8 # maximum length of each sequence

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i + block_size] for i in ix])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[46, 47, 51, 12,  0,  0, 15, 20],
        [58, 53,  1, 59, 57, 43,  1, 47],
        [46,  1, 63, 53, 59,  1, 61, 43],
        [42, 39, 63, 57,  5,  1, 56, 43]])
targets:
torch.Size([4, 8])
tensor([[47, 51, 12,  0,  0, 15, 20, 30],
        [53,  1, 59, 57, 43,  1, 47, 58],
        [ 1, 63, 53, 59,  1, 61, 43, 56],
        [39, 63, 57,  5,  1, 56, 43, 57]])
----
when input is [46] the target: 47
when input is [46, 47] the target: 51
when input is [46, 47, 51] the target: 12
when input is [46, 47, 51, 12] the target: 0
when input is [46, 47, 51, 12, 0] the target: 0
when input is [46, 47, 51, 12, 0, 0] the target: 15
when input is [46, 47, 51, 12, 0, 0, 15] the target: 20
when input is [46, 47, 51, 12, 0, 0, 15, 20] the target: 30
when input is [58] the target: 53
when input is [58, 53] the target: 1
when input is [58, 53, 1] the target: 59
when input is [58, 53, 1, 59] the target: 57
when input is [58, 53, 1, 59, 57] the target: 43
when input is [58, 53

In [22]:
# transformer input
xb 

tensor([[46, 47, 51, 12,  0,  0, 15, 20],
        [58, 53,  1, 59, 57, 43,  1, 47],
        [46,  1, 63, 53, 59,  1, 61, 43],
        [42, 39, 63, 57,  5,  1, 56, 43]])

In [27]:
torch.manual_seed(19)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # each token directly reads off the logits for the next token from a lookup table        
        
    # def forward()
    